In [29]:
import numpy as np
import pandas as pd
from imputation_strategies import *

## CHARGEMENTS DES DONNEES

In [7]:
# Chargement des DataSets
fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')
retail_df = pd.read_csv('../../../Data/Clustering/retail Data/retail_data_clean.csv')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_11632/3500090452.py:2: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')


## SUPPRESSION DES DONNEES

In [8]:
def introduce_missing_values_mcar(df, missing_percentages):
    # Vérifie que les pourcentages sont valides
    if not all(0 <= p <= 1 for p in missing_percentages):
        raise ValueError("Les pourcentages doivent être entre 0 et 1.")

    df_dict = {}

    for percentage in missing_percentages:
        df_missing = df.copy()
        # Pour chaque colonne du DataFrame
        for col in df.columns:
            # Calcule le nombre de valeurs à remplacer par NaN
            n_missing = int(np.floor(percentage * df.shape[0]))
            # Sélectionne aléatoirement les index des valeurs à remplacer
            missing_indices = np.random.choice(df.index, n_missing, replace=False)
            # Remplace les valeurs par NaN
            df_missing.loc[missing_indices, col] = np.nan

        df_dict[f"{int(percentage * 100)}%"] = df_missing

    return df_dict

In [9]:
# Liste des pourcentages de valeurs manquantes à introduire
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8]

# Génération des DataFrames avec valeurs manquantes
fps_df_with_missing = introduce_missing_values_mcar(fps_df, missing_percentages)
heart_df_with_missing = introduce_missing_values_mcar(heart_df, missing_percentages)
retail_df_with_missing = introduce_missing_values_mcar(retail_df, missing_percentages)

In [12]:
# Enregistrement des DataFrames générés
for key, value in fps_df_with_missing.items():
    value.to_csv(f"../../../Data/Regression/fps in video games/Completeness/NaN/fps_{key}.csv", index=False)

for key, value in heart_df_with_missing.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/NaN/heart_{key}.csv", index=False)

for key, value in retail_df_with_missing.items():
    value.to_csv(f"../../../Data/Clustering/retail Data/Completeness/NaN/retail_{key}.csv", index=False)

## Correction des données avec differentes Strategies

In [27]:
def impute_dataset(df, strategy):
    df_imputed = df.copy()
    # Impute numerical columns
    if strategy['numerical'] == 'delete':
        df_imputed = impute_numerical_by_delete(df_imputed)
    elif strategy['numerical'] == 'mean':
        df_imputed = impute_numerical_by_mean(df_imputed)
    elif strategy['numerical'] == 'median':
        df_imputed = impute_numerical_by_median(df_imputed)
    elif strategy['numerical'] == 'knn':
        df_imputed = impute_numerical_by_knn(df_imputed)
    elif strategy['numerical'] == 'multiple':
        df_imputed = impute_numerical_by_multiple(df_imputed)
    elif strategy['numerical'] == 'decision_tree':
        df_imputed = impute_numerical_by_decision_tree(df_imputed)
    else:
        raise ValueError("Unknown numerical imputation strategy")

    # Impute categorical columns
    if strategy['categorical'] == 'delete':
        df_imputed = impute_categorical_by_delete(df_imputed)
    if strategy['categorical'] == 'mode':
        df_imputed = impute_categorical_by_mode(df_imputed)
    elif strategy['categorical'] == 'new':
        df_imputed = impute_categorical_by_new_category(df_imputed)
    elif strategy['categorical'] == 'hot_deck':
        df_imputed = impute_categorical_by_hot_deck(df_imputed)
    elif strategy['categorical'] == 'decision_tree':
        df_imputed = impute_categorical_by_decision_tree(df_imputed)
    else:
        raise ValueError("Unknown categorical imputation strategy")

    return df_imputed

# Example usage:
strategy_delete = {
    'numerical': 'delete',
    'categorical': 'delete'
}

strategy_mean_mode = {
    'numerical': 'mean',
    'categorical': 'mode'
}

strategy_median_new = {
    'numerical': 'median',
    'categorical': 'new'
}

strategy_decision_tree = {
    'numerical': 'decision_tree',
    'categorical': 'decision_tree'
}

strategy_mean_new = {
    'numerical': 'mean',
    'categorical': 'new'
}

strategy_knn_mode = {
    'numerical': 'knn',
    'categorical': 'mode'
}

In [30]:
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8]

# Boucle pour lire les fichiers CSV
for percentage in missing_percentages:
    df = pd.read_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/NaN/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_delete = impute_dataset(df, strategy_delete)
    df_imputed_by_delete.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/Delete lines/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_mean_mode = impute_dataset(df, strategy_mean_mode)
    df_imputed_by_mean_mode.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/Mean and Mode/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_median_new = impute_dataset(df, strategy_median_new)
    df_imputed_by_median_new.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/Median and New/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_decision_tree = impute_dataset(df, strategy_decision_tree)
    df_imputed_by_decision_tree.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/Decision Tree/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_mean_new = impute_dataset(df, strategy_mean_new)
    df_imputed_by_mean_new.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/Mean and New/heart_{int(percentage * 100)}%.csv")
    
    df_imputed_by_knn_mode = impute_dataset(df, strategy_knn_mode)
    df_imputed_by_knn_mode.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/KNN and Mode/heart_{int(percentage * 100)}%.csv")

NameError: name 'impute_numerical_by_delete' is not defined

In [26]:
dataframes[1]

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,NaN,9.0,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,NaN,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,NaN,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,NaN,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",No,...,NaN,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,NaN,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.70,90.72,31.32,No,No,Yes,NaN,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.55,79.38,NaN,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,Virgin Islands,Male,Very good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,Yes,6.0,NaN,No,...,1.78,102.06,32.28,Yes,No,NaN,No,"Yes, received tetanus shot but not sure what type",No,NaN
246018,Virgin Islands,Female,Fair,0.0,7.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,NaN,90.72,24.34,NaN,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
246019,Virgin Islands,Male,Good,NaN,15.0,Within past year (anytime less than 12 months ...,Yes,7.0,1 to 5,No,...,NaN,83.91,29.86,Yes,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",NaN,Yes
246020,Virgin Islands,Female,Excellent,2.0,2.0,Within past year (anytime less than 12 months ...,Yes,NaN,None of them,No,...,1.70,83.01,28.66,No,NaN,Yes,No,"Yes, received tetanus shot but not sure what type",No,No


# Faire modif : ne pas supprimer des valeurs dans la colonne de Target